# Computer Vision Manipulations

In [2]:
import cv2
import numpy as np
import itertools as it
import os
import sys
import multiprocessing as mp
import pandas as pd

## Functions

In [3]:
def createHash(img, bot, top):
    hashmap = {}
    for i,j in it.product(range(img.shape[0]), range(img.shape[1])):
        if img[i,j] not in hashmap:
            if img[i,j] <= bot:
                hashmap[img[i,j]] = 0
            elif img[i,j] > top:
                hashmap[img[i,j]] = 255
            else:
                hashmap[img[i,j]] = 255 * ((img[i,j] - bot) / (top - bot))
                
    return hashmap

def grayscale(img):
    
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [4]:
def linearStretch(img):
    bot = min(img.ravel())
    top = max(img.ravel())
        
    hashmap = createHash(img, bot, top)
    img2 = img
    for i, j in it.product(range(img2.shape[0]), range(img2.shape[1])):
        img2[i,j] = hashmap[img2[i,j]]
        
    return img2

def createROI(img, h1, h2, w1, w2):
    
    return img[h1:h2, w1:w2]

def powerStretch(img):
    img2 = img
    #img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)  
    img2 = img2/255.0      
    im_power_law_transformation = cv2.pow(img2,2.5)
    im_power_law_transformation *= 255 
    im_power_law_transformation = im_power_law_transformation.astype('uint8')
    #im_power_law_transformation = cv2.cvtColor(im_power_law_transformation,cv2.COLOR_BGR2RGB)
        
    return im_power_law_transformation

def hsvStretch(img):
    hsvimg = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)    
    hue = hsvimg[:,:,0]
    saturation = hsvimg[:,:,1]
    value = hsvimg[:,:,2]
    valuecs = linearStretch(value)
    saturationcs = linearStretch(saturation)
    hsvcs = cv2.merge((hue,saturationcs,valuecs))
    img = cv2.cvtColor(hsvcs, cv2.COLOR_HSV2RGB)
   
    return img

def findmax(img):
    maxpixel = img[0,0]
    maxBlue = maxpixel[0]
    maxGreen = maxpixel[1]
    maxRed = maxpixel[2]
    for i,j in it.product(range(img.shape[0]), range(img.shape[1])):
        bluePixel = img[i,j,0]
        greenPixel = img[i,j,1]
        redPixel = img[i,j,2]
        if bluePixel > maxBlue:
            maxBlue = img[i,j,0]
        if greenPixel > maxGreen:
            maxGreen = img[i,j,1]
        if redPixel > maxRed:
            maxRed = img[i,j,2]
            
    return [maxBlue,maxGreen,maxRed]

def rescale(pixel, white):
    val = pixel * white
    if pixel * white > 255:
        return 255 
    return val

def whitebalance(img):
    newwhite = findmax(img)
    scale = [255 / e for e in newwhite]
    b = img[:,:,0]
    g = img[:,:,1]
    r = img[:,:,2]
    hb = {}
    hg = {}
    hr = {}
    for e in list(set(b.ravel())):
        hb[e] = rescale(e, scale[0])
    for e in list(set(g.ravel())):
        hg[e] = rescale(e, scale[1])
    for e in list(set(r.ravel())):
        hr[e] = rescale(e, scale[2])
    newimg = img
    for i,j in it.product(range(img.shape[0]), range(img.shape[1])):
        newimg[i,j] = np.array([hb[img[i,j][0]],hg[img[i,j][1]],hr[img[i,j][2]]])
    return newimg

In [5]:
def getImagesPath(parent_path, child_path, filenames):
    paths = []
    for f in filenames:
        folder = f[:f.find('frame')-1]
        framename = 'wb_' + f[f.find('frame'):]
        if os.path.exists(parent_path + folder + child_path + framename):
            paths.append((folder,framename))
        
    return paths       

In [6]:
parent_path = '/home/diegues/2018-05-01-OMARE/logs/lauv-noptilus-3/20180501/' # change this path to the one you want
child_path = '/mra/FilteredPhotos/WhiteBalance/' # change this path to the one you want

df_path = "/home/diegues/Desktop/ProcessedImages/total_data.csv" # change this path to the one you want
data = pd.read_csv(df_path)

filenames = data['filename']
image_paths = getImagesPath(parent_path, child_path, filenames)
image_paths[:5]

[('104728_cam-np3', 'wb_frame5.jpg'),
 ('104728_cam-np3', 'wb_frame9.jpg'),
 ('104728_cam-np3', 'wb_frame50.jpg'),
 ('104728_cam-np3', 'wb_frame51.jpg'),
 ('104728_cam-np3', 'wb_frame52.jpg')]

In [7]:
cv_path = '/'.join(df_path.split('/')[:-1]) + '/CV/'
wb_path = '/wb/'
hsv_path = '/hsv_cs/'
lcs_path = '/lcs/'
pcs_path = '/pcs/'
if not os.path.exists(cv_path):
    os.mkdir(cv_path)
for folder in [f[0] for f in image_paths]:
    if not os.path.exists(cv_path + folder):
        os.mkdir(cv_path + folder)        
        os.mkdir(cv_path + folder + wb_path)
        os.mkdir(cv_path + folder + hsv_path)
        os.mkdir(cv_path + folder + lcs_path)
        os.mkdir(cv_path + folder + pcs_path)


In [13]:
def createImages(t):
    img = cv2.imread(parent_path + t[0] + child_path + t[1])
    #print('Processing', t)
    wb_img = whitebalance(img)
    hsv_cs = hsvStretch(wb_img)
    lcs = linearStretch(grayscale(img))
    pcs = powerStretch(grayscale(img))
    cv2.imwrite(cv_path + t[0] + wb_path + t[0] + t[1][t[1].find('_'):], wb_img)
    cv2.imwrite(cv_path + t[0] + hsv_path + t[0] + t[1][t[1].find('_'):], hsv_cs)
    cv2.imwrite(cv_path + t[0] + lcs_path + t[0] + t[1][t[1].find('_'):], lcs)
    cv2.imwrite(cv_path + t[0] + pcs_path + t[0] + t[1][t[1].find('_'):], pcs)

In [ ]:
pool = mp.Pool(processes=mp.cpu_count())
pool.map(createImages, image_paths)